<a href="https://colab.research.google.com/github/gabgovar/Apache-Spark/blob/main/Glue_Job.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')
findspark.find()

'spark-2.4.4-bin-hadoop2.7/python/pyspark'

In [29]:
from pyspark.sql.functions import when
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CDC").getOrCreate()

In [45]:
# Banco de dados full
fldf = spark.read.csv("/content/drive/MyDrive/PySpark/data/Pipeline_proj/LOAD00000001.csv")
fldf = fldf.withColumnRenamed("_c0", "id").withColumnRenamed("_c1", "FullName").withColumnRenamed("_c2", "City")
fldf.write.mode("overwrite").csv("/content/drive/MyDrive/PySpark/data/Pipeline_proj/finalOutput/finalFile.csv")

In [46]:
# updated file => updf
updf = spark.read.csv("/content/drive/MyDrive/PySpark/data/Pipeline_proj/20220203-203246530.csv")
updf = updf.withColumnRenamed("_c0", "action").withColumnRenamed("_c1", "id").withColumnRenamed("_c2", "FullName").withColumnRenamed("_c3", "City")
ffdf = spark.read.csv("/content/drive/MyDrive/PySpark/data/Pipeline_proj/finalOutput/finalFile.csv")
ffdf = ffdf.withColumnRenamed("_c0", "id").withColumnRenamed("_c1", "FullName").withColumnRenamed("_c2", "City")
updf.show()

+------+---+-------------+--------+
|action| id|     FullName|    City|
+------+---+-------------+--------+
|     U|  7|      ABC XYZ| Phoenix|
|     I|130|   Alica Bing|New York|
|     I|131| Malinda Bing| Detroit|
|     I|132|Chandler Bing|Portland|
|     U|  8|      ABC XYZ|  Denver|
|     D| 10|   Jack Hicks| Houston|
+------+---+-------------+--------+



In [51]:
ffdf.toPandas()

,id,FullName,City
0,0,Herman Zimmerman,Oklahoma City
1,1,Lisa Ray,Columbus
2,2,Terrell Reeves,Jacksonville
3,3,Steve Goodwin,Charlotte
4,4,Leah Tran,Detroit
...,...,...,...
101,131,Malinda Bing,Detroit
102,132,Chandler Bing,Portland
103,130,Alica Bing,New York
104,131,Malinda Bing,Detroit


In [50]:
for row in updf.collect():
    #Faz o Update
    if (row["action"] == "U"):
        ffdf = ffdf.withColumn("FullName", when(ffdf["id"] == row ["id"], row["FullName"]).otherwise(ffdf["FullName"]))
        ffdf = ffdf.withColumn("City", when(ffdf["id"] == row["id"], row["City"]).otherwise(ffdf["City"]))
    # Faz o Insert
    if (row["action"] == "I"):
        insertedRow = [list(row)[1:]]
        columns = ['id', 'FullName', 'City']
        newdf = spark.createDataFrame(insertedRow, columns)
        ffdf = ffdf.union(newdf)
    # Faz o Delete    
    if (row["action"] == "D"):
        ffdf = ffdf.filter(ffdf.id != row["id"])

# Exporta o arquivo
fldf.write.mode("overwrite").csv("/content/drive/MyDrive/PySpark/data/Pipeline_proj/finalOutput/finalFile.csv")


